In [0]:
# Configura las variables de conexión
jdbcHostname = "**********"
jdbcPort = ****  # Puerto por defecto de SQL Server
jdbcDatabase = "****"
jdbcUsername = "*******"
jdbcPassword = "*******"

# Construye la URL JDBC
jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};databaseName={jdbcDatabase}"

# Opciones de conexión
connectionProperties = {
    "user": jdbcUsername,
    "password": jdbcPassword,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# Consulta SQL
query = """
SELECT [contentId]
      ,[absoluteURL]
      ,[topicName]
FROM [dbo].[ARA_DIM_ARTICLES]
WHERE CAST([publishedDate] AS DATE) BETWEEN '2023-05-01' AND '2024-05-31' 
  AND siteId=4 
  AND topicName IS NOT NULL
  AND contentType NOT IN ('breu', 'video', 'videoembed', 'poll', 'external', 'publication', 'live', 'cartoon', 'photogallery', 'story', 'discussion','podcast')
  AND isBrandedContent='false'
  AND bodyCharacters<8000
  AND [topicName] IN ('Guerra entre Israel i Palestina', 'Eleccions al Parlament', 'L''atac rus a Ucraïna', 'Eleccions municipals', 'Negociacions per a la investidura a l''Estat',
                      'La gran sequera', 'El nou govern de Pedro Sánchez', 'El petó forçat de Luis Rubiales', 'Informe PISA')
"""

# Leer datos desde SQL Server usando la consulta
df = spark.read.jdbc(url=jdbcUrl, table=f"({query}) AS ARA_DIM_ARTICLES", properties=connectionProperties)

from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import json
from newspaper import Article
import re
import time

# Configurar la conexión con Azure Blob Storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=aramlstoragetest;AccountKey=aDjsXo8+Zyd3DdpVLeBwsiOCYAw9kUiXSiPE0/vv+AuR7LinCtriH+xMQB8YTaQZK7WTIqLxNRvY+ASt1x1lBw==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_name = "newsdata"
blob_name = "articles.json"
container_client = blob_service_client.get_container_client(container_name)

# Eliminar el archivo existente si existe
try:
    blob_client = container_client.get_blob_client(blob_name)
    blob_client.delete_blob()
    print(f"Archivo {blob_name} eliminado exitosamente.")
except Exception as e:
    if "BlobNotFound" in str(e):
        print(f"El archivo {blob_name} no existe. Continuando con el proceso...")
    else:
        print(f"No se pudo eliminar el archivo {blob_name}: {e}")

# Función para limpiar el texto
def preprocess_text(text):
    # Reemplazar múltiples espacios por un solo espacio
    text = re.sub(r'\s+', ' ', text)
    # Insertar espacio después de palabras concatenadas sin espacio
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    # Eliminar frases que comienzan con "Inscriu-te" y terminan con "Inscriu-t’hi"
    text = re.sub(r'Inscriu-te.*?Inscriu-t’hi', '', text)
    return text

# Función para descargar el contenido de una URL con reintentos
def download_article(url, retries=5, backoff_factor=0.3):
    for i in range(retries):
        try:
            article = Article(url)
            article.download()
            article.parse()
            cleaned_text = preprocess_text(article.text)
            return {
                'url': url,
                'text': cleaned_text
            }
        except Exception as e:
            print(f"Error descargando {url}: {e}")
            if i < retries - 1:  # No esperar después del último intento
                wait_time = backoff_factor * (2 ** i)  # Espera exponencial
                print(f"Reintentando en {wait_time} segundos...")
                time.sleep(wait_time)
    return None  # Retorna None si todos los intentos fallan

# Descargar las noticias y preprocesar el texto
articles_data = []
urls = df.select("absoluteURL", "contentId", "topicName").collect()

for row in urls:
    url = row['absoluteURL']
    content_id = row['contentId']
    topic_name = row['topicName']
    article = download_article(url)
    if article:
        article['contentId'] = content_id
        article['topicName'] = topic_name
        articles_data.append(article)

# Convertir a JSON
articles_json = json.dumps(articles_data, ensure_ascii=False, indent=4)

# Crear BlobClient
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Subir el JSON al Blob Storage
blob_client.upload_blob(articles_json, blob_type="BlockBlob", overwrite=True)

# Mensaje final con el número de noticias guardadas
print(f"Noticias guardadas exitosamente en Azure Blob Storage. Total de noticias guardadas: {len(articles_data)}")


Archivo articles.json eliminado exitosamente.
Noticias guardadas exitosamente en Azure Blob Storage. Total de noticias guardadas: 2100